In [ ]:
#import required libraries
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import re
import pandas as pd
import urllib.request

## Q1

In [ ]:
def search(term="Laptop"):
    search_xpath = '//*[@id="twotabsearchtextbox"]'
    search_button = wait.until(ec.element_to_be_clickable((By.XPATH, search_xpath)))
    search_button.send_keys(term)
    search_button.send_keys(Keys.ENTER)

if __name__=="__main__":
    term = input("Enter product : ")
    browser = webdriver.Chrome()
    browser.get("https://www.amazon.in")
    wait = WebDriverWait(browser, 600)
    search(term)

## Q2

In [ ]:
def search(term="Laptop"):
    search_xpath = '//*[@id="twotabsearchtextbox"]'
    search_button = wait.until(ec.element_to_be_clickable((By.XPATH, search_xpath)))
    search_button.send_keys(term)
    search_button.send_keys(Keys.ENTER)

def getProducts():
    products = soup.find_all("a", {"class":"a-link-normal s-no-outline"})
    if len(products)>0:
        return products
    return []

def clickNext(browser):
    xpath = '//li[@class="a-last"]'
    btn = wait.until(ec.element_to_be_clickable((By.XPATH, xpath)))
    btn.click()

def brandName(soup):
    try:
        return soup.find(text="Brand").findNext("td").get_text().strip()
    except:
        return "-"

def prodName(soup):
    try:
        data = soup.find("span",{"id":"productTitle"}).get_text()
        data = re.sub("\s+"," ",data)
        return data.strip()
    except:
        return "-"

def getRating(soup):
    try:
        data = soup.find("span",{"class":"a-icon-alt"}).get_text().split(" ")[0]
        if "Previous" in data:
            return "-"
        else:
            return data
    except:
        return "-"

def numRatings(soup):
    try:
        return soup.find("span",{"id":"acrCustomerReviewText"}).get_text().split(" ")[0]
    except:
        return "-"

def getPrice(soup):
    try:
        return soup.find("span",{"id":"priceblock_ourprice"}).get_text()
    except:
        return "-"

def returnExchange(soup):
    return bool(soup.find("a",{"class":"a-size-small a-link-normal a-text-normal"}).get_text())

def expectedDelivery(soup):
    try:
        div = soup.find("div",{"id":"ddmDeliveryMessage"})
        return div.find("b").get_text().strip()
    except:
        return "-"

def availability(soup):
    text = soup.get_text().lower()
    return bool(re.search("in stock", text))

def otherDetails(soup):
    try:
        data = soup.find("div",{"id":"productDescription"}).get_text()
        data = re.sub("\s+"," ",data)
        return data.strip()
    except:
        return "-"

def report(brand_name, prod_name, rating, num_of_ratings, price, return_exchange, expected_delivery, avail, other_details,prod_url):
    df = pd.read_excel("DataQ2.xlsx")

    #Saving data into excel sheet
    data = {"Brand Name":[brand_name], "Product Name":[prod_name],"Rating":[rating],
            "No. of Ratings":[num_of_ratings],"Price":[price],"Return/Exchange":[return_exchange],
            "Expected Delivery":[expected_delivery],"Availability":[avail],
            "Other Details":[other_details],"Product URL":[prod_url]
            }
    df = df.append(pd.DataFrame(data))
    writer = pd.ExcelWriter("DataQ2.xlsx")
    df.to_excel(writer,index=False)
    writer.save()


if __name__=="__main__":
    term = input("Enter product : ")
    browser = webdriver.Chrome()
    browser.get("https://www.amazon.in")
    wait = WebDriverWait(browser, 600)
    search(term)

    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")

    products = []
    for i in range(3):
        products=products+getProducts()
        try:
            clickNext(browser)
        except:
            continue

    for product in products:
        prod_url = "https://www.amazon.in"+product["href"]
        browser.get(prod_url)
        html = browser.page_source
        soup = BeautifulSoup(html,"html.parser")

        brand_name = brandName(soup)
        prod_name = prodName(soup)
        rating = getRating(soup)
        num_of_ratings = numRatings(soup)
        price = getPrice(soup)
        return_exchange = returnExchange(soup)
        expected_delivery = expectedDelivery(soup)
        avail = availability(soup)
        other_details = otherDetails(soup)
        """
        print(brand_name)
        print(prod_name)
        print(rating)
        print(num_of_ratings)
        print(price)
        print(return_exchange)
        print(expected_delivery)
        print(avail)
        print(other_details)
        """
        report(brand_name, prod_name, rating, num_of_ratings, price, return_exchange, expected_delivery, avail, other_details, prod_url)

## Q3

In [ ]:
def search(query):
    search_xpath = '//input[@class="gLFyf gsfi"]'
    search_button = wait.until(ec.element_to_be_clickable((By.XPATH, search_xpath)))
    search_button.send_keys(query)
    search_button.send_keys(Keys.ENTER)

def getURLs(soup):
    div = soup.find("div",{"class":"islrc"})
    imgs = div.find_all("img",limit=100)
    urls = []
    for img in imgs:
        urls.append(img["src"])
    return urls

if __name__=="__main__":
    browser = webdriver.Chrome()
    wait = WebDriverWait(browser, 600)
    queries = ["fruits", "cars", "Machine Learning"]

    for query in queries:
        print("Now searching for {}".format(query))
        browser.get("https://images.google.com/")
        search(query)
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")
        urls = getURLs(soup)
        for i,url in enumerate(urls):
            file_name = "./Q3-images/"+query+"/"+str(i)+".jpg"
            urllib.request.urlretrieve(url, file_name)

## Q4

In [ ]:
def brandName(soup):
    try:
        return soup.find("span",{"class":"B_NuCI"}).get_text().strip().split()[0]
    except:
        return "-"

def phoneName(soup):
    try:
        return soup.find("span",{"class":"B_NuCI"}).get_text().strip()
    except:
        return "-"

def color(soup):
    try:
        return soup.find(text="Color").findNext("td").get_text().strip()
    except:
        return "-"

def ram(soup):
    try:
        return soup.find(text="Ram Capacity").findNext("td").get_text().strip()
    except:
        try:
            return soup.find(text="RAM").findNext("td").get_text().strip()
        except:
            return "-"

def storage(soup):
    try:
        return soup.find(text="Storage Memory").findNext("td").get_text().strip()
    except:
        try:
            return soup.find(text="Internal Storage").findNext("td").get_text().strip()
        except:
            return "-"

def primaryCamera(soup):
    try:
        return soup.find(text="Primary Camera").findNext("td").get_text().strip()
    except:
        return "-"

def secondCamera(soup):
    try:
        return soup.find(text="Secondary Camera").findNext("td").get_text().strip()
    except:
        return "-"

def displaySize(soup):
    try:
        return soup.find(text="Display Size").findNext("td").get_text().strip()
    except:
        return "-"

def displayResol(soup):
    try:
        return soup.find(text="Resolution").findNext("td").get_text().strip()
    except:
        return "-"

def processor(soup):
    try:
        return soup.find(text="Processor Type").findNext("td").get_text().strip()
    except:
        return "-"

def cores(soup):
    try:
        return soup.find(text="Processor Core").findNext("td").get_text().strip()
    except:
        return "-"

def battery(soup):
    try:
        return soup.find(text="Battery Capacity").findNext("td").get_text().strip()
    except:
        return "-"

def price(soup):
    try:
        return soup.find("div",{"class":"_30jeq3 _16Jk6d"}).get_text().strip()
    except:
        return "-"

def report(brand, name, col, rm, rom, pri_cam, sec_cam, dis_size,
                des_res, proc, core, bat, ruppee, prod_url):
    df = pd.read_excel("Q4.xlsx")
    #Saving data into excel sheet
    data = {"Brand Name":[brand], "Product Name":[name],"Color":[col],
            "RAM":[rm],"ROM":[rom],"Primary Camera":[pri_cam],
            "Secondary Camera":[sec_cam],"Display Size":[dis_size],
            "Display Resolution":[des_res],"Processor":[proc],"Cores":[core],
            "Battery":[bat],"Price":[ruppee],"Product URL":[prod_url]
            }
    df = df.append(pd.DataFrame(data))
    writer = pd.ExcelWriter("Q4.xlsx")
    df.to_excel(writer,index=False)
    writer.save()


if __name__=="__main__":
    browser = webdriver.Chrome()
    browser.get("https://www.flipkart.com")
    wait = WebDriverWait(browser, 600)

    #close button
    xpath = '//button[@class="_2KpZ6l _2doB4z"]'
    btn = browser.find_element_by_xpath(xpath)
    try:
        btn.click()
    except:
        pass

    #search
    xpath = '//input[@title="Search for products, brands and more"]'
    btn = browser.find_element_by_xpath(xpath)
    btn.send_keys("Oneplus Nord")
    btn.send_keys(Keys.ENTER)

    #get all a's
    time.sleep(5)
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")

    a_tags = soup.find_all("a",{"class":"_1fQZEK"})
    urls = []
    for a in a_tags:
        prod_url = "https://www.flipkart.com"+a["href"]
        urls.append(prod_url)

    #Iterate through each url
    for url in urls:
        browser.get(url)
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")

        brand = brandName(soup)
        name = phoneName(soup)
        col = color(soup)
        rm = ram(soup)
        rom = storage(soup)
        pri_cam = primaryCamera(soup)
        sec_cam = secondCamera(soup)
        dis_size = displaySize(soup)
        des_res = displayResol(soup)
        proc= processor(soup)
        core = cores(soup)
        bat = battery(soup)
        ruppee = price(soup)

        report(brand, name, col, rm, rom, pri_cam, sec_cam, dis_size,
                des_res, proc, core, bat, ruppee, prod_url)

## Q7

In [ ]:
if __name__=="__main__":
    browser = webdriver.Chrome()
    wait = WebDriverWait(browser, 600)

    #Goto Homepage
    browser.get("https://www.digit.in/")

    #Goto Gaming Tab
    browser.get("https://www.digit.in/gaming/")

    #Goto Gaming mart tab
    browser.get("https://www.digit.in/gaming-mart/")

    #Goto Gaming Laptops
    browser.get("https://www.digit.in/gaming-mart/topten/gaming-laptops/")

    #Click on Read more button
    xpath = '//a[@class="review"]'
    btn = wait.until(ec.element_to_be_clickable((By.XPATH, xpath)))
    btn.click()

    time.sleep(3)

    html = browser.page_source
    soup = BeautifulSoup(html,"html.parser")

    #Get all product links
    urls = soup.find_all("a",{"class":"review spec"})

    for i,url in enumerate(urls,start=1):
        print("Getting data for product {}".format(i))
        print()
        browser.get(url["href"])
        time.sleep(3)
        html = browser.page_source
        soup = BeautifulSoup(html,"html.parser")

        details = soup.find("div",{"class":"tabcontent"}).get_text().strip()
        details = re.sub("\s\s+","\n",details)
        details = re.sub("\s:\s",":",details)

        print(details)
        print()
        print()

## Q8

In [ ]:
def report(rank, name, net_worth, age, citizen, source, industry):
    df = pd.read_excel("Q8.xlsx")
    #Saving data into excel sheet
    data = {"Rank":[rank],"Name":[name],"Net Worth":[net_worth],
            "Age":[age],"Citizenship":[citizen],"Source":[source],
            "Industry":[industry]
            }
    df = df.append(pd.DataFrame(data))
    writer = pd.ExcelWriter("Q8.xlsx")
    df.to_excel(writer,index=False)
    writer.save()

if __name__ == "__main__":
    browser = webdriver.Chrome()
    wait = WebDriverWait(browser, 600)

    browser.get("https://www.forbes.com")
    browser.execute_script("window.close();")

    #Goto Billionaires list
    browser.get("https://www.forbes.com/billionaires/")
    time.sleep(3)

    ranks = browser.find_elements_by_class_name("rank")
    names = browser.find_elements_by_class_name("personName")
    nets = browser.find_elements_by_class_name("netWorth")
    ages = browser.find_elements_by_class_name("age")
    citizens = browser.find_elements_by_class_name("countryOfCitizenship")
    sources = browser.find_elements_by_class_name("source")
    categories = browser.find_elements_by_class_name("category")

    for rank,name,net,age,citizen,source,category in zip(ranks,names,nets,ages,citizens,sources,categories):
        print(rank.text, name.text, net.text, age.text, citizen.text, source.text, category.text)
        print()
        print()
        report(rank.text, name.text, net.text, age.text, citizen.text, source.text, category.text)